In [ ]:
pip install pandas tqdm openai PyMuPDF

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
from io import StringIO
import fitz 
import csv


In [ ]:
# Initialize DeepSeek client
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("API key not found. Please set the DEEPSEEK_API_KEY environment variable.")

client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com"  # DeepSeek API base URL
)


# Define columns
COLUMNS = [
    "Catalogue Identifier", "City", "Year", "Duration", "Type",
    "Institution", "Catalogue Price", "Monday Open",
    "Monday Time", "Monday Lowest Price", "Monday Highest Price", "Tuesday Open", "Tuesday Time",
    "Tuesday Lowest Price", "Tuesday Highest Price", "Wednesday Open", "Wednesday Time", "Wednesday Lowest Price", 
    "Wednesday Highest Price", "Thursday Open", "Thursday Time", "Thursday Lowest Price", "Thursday Highest Price", "Friday Open", 
    "Friday Time", "Friday Lowest Price", "Friday Highest Price", "Saturday Open", "Saturday Time", 
    "Saturday Lowest Price", "Saturday Highest Price", "Sunday Open", "Sunday Time", "Sunday Lowest Price",
    "Sunday Highest Price", "Membership Available", "Membership Price", "Membership Benefits", "Lottery",
    "Lottery price", "Competition", "Minimum Age", "Total Entries", "misc", "Reasoner"
]


# Extract the text from the PDFs
def extract_text_from_pdf(pdf_path):
    try:
        text = ""
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text()  
        return text.strip()
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return ""

# Prompt the LLM to extract, interpret and structure the data
def format_text_into_table(raw_text, pdf_name):
   
   
    catalogue_identifier = pdf_name


    try:
        prompt = f"""These are Dutch exhibition catalogs of 'levende meesters'. You are tasked with finding, extracting and structuring exhibition information from each catalog into a CSV table:
        {raw_text}
        
        Rules:

        Mandatory columns:
        - **Catalogue Identifier:** Use {catalogue_identifier} for every row. ALWAYS do this, NEVER SKIP.
        - **City:** Identify where the exhibition was held. This is usually found on the front page.
        - **Year:** Extract the year of the exhibition.
        - **Duration:** Extract for how long the exhibition is held in dd/mm-dd/mm format.
        - **Type:** Identify whether it is a national or international exhibition
        - **Institution:** Extract the name of the institution. (e.g. musis sacrum, pictura, arti et amicitiae)
        - **Catalogue Price:** Extract the catalog price (e.g., ƒ0.25). Avoid entry prices.
        - **[Day] Open:
             If the exhibition is open on a given day, set "true".
             If it is closed, set "false".
             If the exhibition is open on that day some weeks but closed on others, set "modified".
             If there is no information available, set "unknown".
             assume that phrases like "en volgende dagen" imply that every day in the week is open
             Important: Consider that date references may indicate a schedule change. Ensure that opening status reflects these adjustments correctly.
        - **[Day] Time:
            Store opening times in HH:MM-HH:MM format (e.g., 10:00-17:00).
            If times are incomplete (e.g., "10-5"), standardize to "10:00-17:00".
            If a day has multiple time ranges e.g. a break between opening times store a range using the earliest time and the latest time. 
            (e.g. "geopend des vooremiddags van 10 tot 2 uren , en des nademiddags van 3 tot 5 uren" means a range of 10:00-17:00). 
            Do the same when a day has multiple opening times throughout the exhibition.
        - **[Day] Lowest Price:** Store lowest entry price for that day (e.g., "0.25"). If no info, store "unknown".**
        - **[Day] Highest Price:** Store Highest possible entry price for that day (e.g. "0.50"). If no info, store "unknown".**
        - **If price equates to free entry, store as 0**
        - **If there is no price deviation, store the same price in both columns. Do not confuse entry price for catalogue price.**
        - **If the exhibition is not open on a certain day, Assume that the time and price columns for that day should be filled with "unavailable" rather than "unknown"**
        - **Minimum Age:** If an indicator for minimum age is present in the catalogue store here, otherwise store "unknown"
        - **Membership Available:** If introductions mention Memberships ("abonnement") store as true, if not store as false.
        - **Membership Price:** Store the membership price mentioned. 
            Do NOT confuse this with Catalog Price, Entry Price or lottery ticket price if lottery is not included in membership. 
            If there are multiple membership prices, store an adjusted range that takes the cheapest price and the most expensive price.
        - **Membership Benefits:** If benefits are mentioned, summarize them in Dutch (do not translate). Examples: "gratis toegang", "deelname aan een loterij".
        - **benefit examples: free entry or a being entered into a raffle
        - **Lottery:** If a lottery is being held, store "true", otherwise "false".
        - **Lottery price:** store the price of lottery participation. 
            If there are multiple lottery (ticket) prices, store an adjusted range that takes the cheapest price and the most expensive price.
        - **Competition: If a competition is mentioned, store "true", otherwise "false".
            If words like "mededinging", "jury" "GK", or "MKM" appear, assume a competition is being held.
        - **Total Entries:** a long list of artwork entries follows the introductory pages. Find the end of the list and store the last Entry number.
        - **misc:** If there are more entry conditions that do not fit in any other category store here. Keep in original dutch
          - examples are rules about entry conditions, touching the artworks or umbrella storage, if membership only applies to "vereniging" with diploma or is open to all consumers etc.
        - **Reasoner:** Use this column to store reasoning steps for each row. 

        processing:
        - **Ensure all data is formatted consistently across rows.**
        - **Use "ƒ" for guilders as currency when applicable.**
        - **Fill missing values with "unknown".**
        - **Leave no empty columns**
        - **Do not skip any pdfs**
        - **Do not add new, remove or adjust the given column headers.**
           

        {','.join(COLUMNS)}
        """

        response = client.chat.completions.create(
            model="deepseek-reasoner", # load preferred model
            messages=[
                {"role": "system", "content": "You are a data formatting assistant. Output CSV data only."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,
            max_tokens=4000,
            stream=False
        )

        if response.choices[0].message.content:
            formatted_data = response.choices[0].message.content.strip()
            df = pd.read_csv(StringIO(formatted_data), delimiter=",", names=COLUMNS, quotechar='"', on_bad_lines="skip")
            return df
    except Exception as e:
        print(f"Error formatting text from {pdf_name}: {e}")
    return pd.DataFrame(columns=COLUMNS)

# Post-processing the output
def adjust_csv_after_creation(csv_file_path):
    try:
        df = pd.read_csv(csv_file_path)

        # Remove duplicate headers and clean strings
        df = remove_duplicate_headers(df)

        # Drop completely empty rows
        df.dropna(how='all', inplace=True)

        # Fill missing values
        df.fillna("unknown", inplace=True)

        df.to_csv(csv_file_path, index=False)
        print(f"CSV adjusted and saved to {csv_file_path}")

    except Exception as e:
        print(f"Error adjusting the CSV: {e}")

def remove_duplicate_headers(df):
    header_values = [str(col).lower() for col in df.columns]
    
    # Remove duplicate header rows
    df = df[~df.apply(lambda row: list(row.astype(str).str.lower()) == header_values, axis=1)]
    
    # Strip whitespace from all string entries 
    str_cols = df.select_dtypes(include=["object"]).columns
    df[str_cols] = df[str_cols].apply(lambda col: col.str.strip())
    
    return df

def process_pdfs(base_folder):
    
    extracted_data = []

    for pdf_file in tqdm(os.listdir(base_folder), desc="Processing PDFs"):
        if pdf_file.lower().endswith(".pdf"):
            pdf_path = os.path.join(base_folder, pdf_file)
            print(f"Processing PDF: {pdf_file}")

            # Extract raw text from PDF
            raw_text = extract_text_from_pdf(pdf_path)
            if not raw_text:
                print(f"No text extracted from {pdf_file}")
                continue

            # Format text into structured table
            df_entries = format_text_into_table(raw_text, pdf_file)
            if not df_entries.empty:
                extracted_data.append(df_entries)

    # Ensure correct CSV writing
    output_csv_path = "output.csv"
    with open(output_csv_path, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
    
        # Write header only once
        writer.writerow(COLUMNS)

        for df in extracted_data:
            for _, row in df.iterrows():
                cleaned_row = [str(value).replace("\n", " ").strip() for value in row]
                writer.writerow(cleaned_row)

    # Adjust the CSV after creation
    adjust_csv_after_creation(output_csv_path)

    # Combine all data into one DataFrame
    final_df = pd.concat(extracted_data, ignore_index=True) if extracted_data else pd.DataFrame(columns=COLUMNS)

    # Remove duplicate headers and clean strings
    final_df = remove_duplicate_headers(final_df)

    # Save to Excel
    if not final_df.empty:
        output_file = "___" # Insert output file title
        final_df.to_excel(output_file, index=False)
        print(f"Extraction complete! Data saved to {output_file}")
    else:
        print("No data extracted! Check API responses.")

# Run the process
if __name__ == "__main__":
    BASE_FOLDER = r"___" # Insert input file path
    process_pdfs(BASE_FOLDER)